In [1]:
import os
import json

from credentials import OPENAPI_KEY
os.environ["OPENAI_API_KEY"] = OPENAPI_KEY

import openai
from tqdm import tqdm
import pandas as pd

In [2]:
client = openai.OpenAI()

In [3]:
ROOT_PATH = "PATH"
ds_name = "hatespeechexistence"
ds_path = f"{ROOT_PATH}/{ds_name}.jsonl"
with open(ds_path, "r") as f:
    my_dataset = [json.loads(line) for line in f]

In [5]:
my_dataset[0]

{'messages': [{'role': 'system',
   'content': "VRL-Detector, verilen Türkçe metnin Nefret Söylemi içerip içermediğini tespit edebilen bir chatbot'tur. (Evet, Hayır)"},
  {'role': 'user',
   'content': 'Bu Türkçe metin nefret söylemi içeriyor mu?:" suriyeli çetelerin evlilik vaadiyle kandırıp binlerce tl dolandırılan cahillere zerre kadar üzülmüyorum ...türkiye\'de kadın kalmadı de mi? beter olun... 15 ocak çarşamba" (Evet, Hayır)'},
  {'role': 'assistant', 'content': 'Evet'}]}

In [5]:
preds = []
refs  = []

for example in tqdm(my_dataset):
    response = client.chat.completions.create(
        model="gpt-4o",  # or "gpt-4-turbo"
        messages=example["messages"][:2]
    )
    
    model_output = response.choices[0].message.content
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output)
    refs.append(ground_truth)

100%|██████████| 5854/5854 [58:36<00:00,  1.85it/s]


In [ ]:
output_dir = f"{ROOT_PATH}/outputs"
output_df = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv(f"{output_dir}/gpt4o-{ds_name}.csv", index=False)

In [4]:
output_df["preds"].value_counts()

preds
Evet.                                                                                                                                                            3508
Hayır.                                                                                                                                                           1381
Evet                                                                                                                                                              800
Hayır                                                                                                                                                             161
Evet. \n\nMetin, Suriyelilere yönelik olumsuz ve düşmanca bir dil kullanmakta ve onları "nankör" ve "işgalci" olarak niteleyerek nefret söylemi içermektedir.       1
Bu metin nefret söylemi içermiyor. (Hayır)                                                                                                                          